In [1]:
import pandas as pd
import numpy as np
import os
from datetime import datetime
from source.TrainDataGeneration import append_ORD_TM,append_ORD_VOL,append_TM_GP,GetGroupDataFrame,append_STEP5
from DataInfo import ISU_list_MBRN_sum_top,MBRN_sum_top

In [2]:
ISU_list=ISU_list_MBRN_sum_top[:33]
ISU_list=[ISU_CD for ISU_CD in ISU_list if ISU_CD not in ['KR7035720002','KR7008800005', 'KR7014940001']]
ISU_list=['KR7071930002',
 'KR7040670002',
 'KR7044180008',
 'KR7052420007',
 'KR7043100007',
 'KR7024810004',
 'KR7036540003',
 'KR7058820002',
 'KR7018700005',
 'KR7043200005']
Dataname='1516Train'
groupmin=10

In [3]:
len(ISU_list)

10

In [4]:
ISU_list

['KR7071930002',
 'KR7040670002',
 'KR7044180008',
 'KR7052420007',
 'KR7043100007',
 'KR7024810004',
 'KR7036540003',
 'KR7058820002',
 'KR7018700005',
 'KR7043200005']

In [5]:
a=os.listdir('/Data/LOBData/CollectedTickData/ISUTickData1516/')

In [6]:
a


['GDF_tick_KR7018700005_10.csv',
 'GDF_tick_KR7071930002_10.csv',
 'GDF_tick_KR7044180008_10.csv',
 'GDF_tick_KR7052420007_10.csv',
 'GDF_tick_KR7043100007_10.csv',
 'GDF_tick_KR7043200005_10.csv',
 'GDF_tick_KR7024810004_10.csv',
 'GDF_tick_KR7058820002_10.csv',
 'GDF_tick_KR7036540003_10.csv',
 'GDF_tick_KR7040670002_10.csv']

In [7]:
groupcolumns=['ORD_DD','ISU_CD','TM_GP']
sumcolumns=[]


meancolumns=['호가장처리가격','ASK_STEP1_BSTORD_PRC','ASK_STEP1_BSTORD_RQTY', 'ASK_STEP2_BSTORD_PRC','ASK_STEP2_BSTORD_RQTY', 
             'ASK_STEP3_BSTORD_PRC','ASK_STEP3_BSTORD_RQTY', 'ASK_STEP4_BSTORD_PRC','ASK_STEP4_BSTORD_RQTY', 
             'ASK_STEP5_BSTORD_PRC','ASK_STEP5_BSTORD_RQTY', 'BID_STEP1_BSTORD_PRC','BID_STEP1_BSTORD_RQTY', 
             'BID_STEP2_BSTORD_PRC','BID_STEP2_BSTORD_RQTY', 'BID_STEP3_BSTORD_PRC','BID_STEP3_BSTORD_RQTY', 
             'BID_STEP4_BSTORD_PRC','BID_STEP4_BSTORD_RQTY', 'BID_STEP5_BSTORD_PRC','BID_STEP5_BSTORD_RQTY', 
             '매도5단계호가합계잔량','매수5단계호가합계잔량','매도10단계호가합계잔량', '매수10단계호가합계잔량',
             '매도총호가잔량','매수총호가잔량','ORD_PRC', '시가','고가','저가',
             '누적체결수량', '누적거래대금','직전체결가격']
#lastcolumns=[column for column in columns if column not in (sumcolumns+meancolumns+groupcolumns)]
lastcolumns=[]

In [8]:
# TickDataname='GDF2022_5_4_8_25.csv'
ISU_CD=ISU_list[0]
TickDataname="GDF_tick"+"_"+ISU_CD+'_'+str(groupmin)+'.csv'
Data=pd.read_csv('/Data/LOBData/CollectedTickData/ISUTickData1516/'+TickDataname)
Data=append_ORD_TM(Data)
Data=append_ORD_VOL(Data)
Data=append_TM_GP(Data,groupmin=groupmin)
Data=append_STEP5(Data)
GDF=GetGroupDataFrame(Data,groupcolumns,sumcolumns,meancolumns,lastcolumns)
TickGDF=GDF

for ISU_CD in ISU_list[1:]:
    TickDataname="GDF_tick"+"_"+ISU_CD+'_'+str(groupmin)+'.csv'
    print(TickDataname)
    Data=pd.read_csv('/Data/LOBData/CollectedTickData/ISUTickData1516/'+TickDataname)
    Data=append_ORD_TM(Data)
    Data=append_ORD_VOL(Data)
    Data=append_TM_GP(Data,groupmin=groupmin)
    Data=append_STEP5(Data)
    GDF=GetGroupDataFrame(Data,groupcolumns,sumcolumns,meancolumns,lastcolumns)
    TickGDF=pd.concat([TickGDF,GDF],axis=0)

GDF_tick_KR7040670002_10.csv
GDF_tick_KR7044180008_10.csv


/root/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3457: DtypeWarning: Columns (9) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


GDF_tick_KR7052420007_10.csv
GDF_tick_KR7043100007_10.csv
GDF_tick_KR7024810004_10.csv
GDF_tick_KR7036540003_10.csv
GDF_tick_KR7058820002_10.csv
GDF_tick_KR7018700005_10.csv
GDF_tick_KR7043200005_10.csv


In [9]:
for col in TickGDF.columns:
    print(col)

ORD_DD
ISU_CD
TM_GP
호가장처리가격
ASK_STEP1_BSTORD_PRC
ASK_STEP1_BSTORD_RQTY
ASK_STEP2_BSTORD_PRC
ASK_STEP2_BSTORD_RQTY
ASK_STEP3_BSTORD_PRC
ASK_STEP3_BSTORD_RQTY
ASK_STEP4_BSTORD_PRC
ASK_STEP4_BSTORD_RQTY
ASK_STEP5_BSTORD_PRC
ASK_STEP5_BSTORD_RQTY
BID_STEP1_BSTORD_PRC
BID_STEP1_BSTORD_RQTY
BID_STEP2_BSTORD_PRC
BID_STEP2_BSTORD_RQTY
BID_STEP3_BSTORD_PRC
BID_STEP3_BSTORD_RQTY
BID_STEP4_BSTORD_PRC
BID_STEP4_BSTORD_RQTY
BID_STEP5_BSTORD_PRC
BID_STEP5_BSTORD_RQTY
매도5단계호가합계잔량
매수5단계호가합계잔량
매도10단계호가합계잔량
매수10단계호가합계잔량
매도총호가잔량
매수총호가잔량
ORD_PRC
시가
고가
저가
누적체결수량
누적거래대금
직전체결가격


In [10]:
TGDF=TickGDF.set_index(['ORD_DD','ISU_CD','TM_GP'])
new_index = pd.MultiIndex.from_product(TGDF.index.levels)
new_df = TGDF.reindex(new_index)
new_df = new_df.ffill()

new_df=new_df.reset_index()
new_df=new_df[(new_df['TM_GP']>=0)&(new_df['TM_GP']<39)]
new_df=new_df.set_index(['ORD_DD','ISU_CD','TM_GP'])

new_index = pd.MultiIndex.from_product(new_df.index.levels)
new_df=new_df.reset_index()

Train_df=new_df
Train_df.loc[new_df['TM_GP']==0.,'시가']=Train_df.loc[new_df['TM_GP']==1.,'시가'].values

In [11]:
Train_df

,ORD_DD,ISU_CD,TM_GP,호가장처리가격,ASK_STEP1_BSTORD_PRC,ASK_STEP1_BSTORD_RQTY,ASK_STEP2_BSTORD_PRC,ASK_STEP2_BSTORD_RQTY,ASK_STEP3_BSTORD_PRC,ASK_STEP3_BSTORD_RQTY,...,매수10단계호가합계잔량,매도총호가잔량,매수총호가잔량,ORD_PRC,시가,고가,저가,누적체결수량,누적거래대금,직전체결가격
0,20150601,KR7018700005,0.0,764.558824,759.622549,3.217387e+03,761.392157,6307.411765,762.730392,6981.828431,...,71164.705882,6.341371e+05,354325.049020,723.901961,762.0,796.264706,749.514706,836.666667,6.356706e+05,758.186275
1,20150601,KR7018700005,1.0,744.210084,741.613445,8.523895e+04,743.336134,23937.075630,744.596639,6167.151261,...,74699.596639,6.523387e+05,174601.731092,695.336134,762.0,797.000000,732.949580,1648.319328,1.215550e+06,741.226891
2,20150601,KR7018700005,2.0,738.924370,729.462185,1.594209e+05,731.579832,5576.226891,732.621849,4160.218487,...,71353.848739,7.429386e+05,71353.848739,695.815126,762.0,797.000000,727.000000,1117.033613,8.143224e+05,729.436975
3,20150601,KR7018700005,3.0,731.469484,727.126761,1.422302e+06,728.248826,11210.802817,729.248826,2467.173709,...,1295.009390,2.100031e+06,1295.009390,718.826291,762.0,797.000000,727.000000,266.732394,1.939907e+05,727.122066
4,20150601,KR7018700005,4.0,727.269231,727.000000,1.657631e+06,728.000000,11533.591512,729.000000,204.082228,...,0.000000,2.399255e+06,0.000000,726.305040,762.0,797.000000,727.000000,10.094164,7.338458e+03,727.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57715,20151230,KR7071930002,34.0,2106.666667,2118.333333,7.366667e+01,2123.333333,43.333333,2128.333333,10.000000,...,1922.000000,6.443333e+03,9876.666667,2106.666667,0.0,2115.000000,2100.000000,66.666667,1.406667e+05,2115.000000
57716,20151230,KR7071930002,35.0,2106.666667,2118.333333,7.366667e+01,2123.333333,43.333333,2128.333333,10.000000,...,1922.000000,6.443333e+03,9876.666667,2106.666667,0.0,2115.000000,2100.000000,66.666667,1.406667e+05,2115.000000
57717,20151230,KR7071930002,36.0,2106.666667,2118.333333,7.366667e+01,2123.333333,43.333333,2128.333333,10.000000,...,1922.000000,6.443333e+03,9876.666667,2106.666667,0.0,2115.000000,2100.000000,66.666667,1.406667e+05,2115.000000
57718,20151230,KR7071930002,37.0,2106.666667,2118.333333,7.366667e+01,2123.333333,43.333333,2128.333333,10.000000,...,1922.000000,6.443333e+03,9876.666667,2106.666667,0.0,2115.000000,2100.000000,66.666667,1.406667e+05,2115.000000


In [12]:
Train_df['10단계호가합계잔량']=Train_df['매수10단계호가합계잔량']+Train_df['매도10단계호가합계잔량']

In [13]:
divdict={}
# divdict['10단계호가합계잔량']=qtylist
# divdict['시가']=['고가','저가','직전체결가격']
loglist=[]
loglist+=['고가','저가','직전체결가격']
# loglist+=['고가','저가','직전체결가격','시가']
# loglist+=['10단계호가합계잔량']
minmaxnormlist=[]
minmaxnormlist+=['매도총호가잔량', '매수총호가잔량', '고가', '저가', 
           '직전체결가격']

In [14]:
for divcol in divdict:
    Train_df[divdict[divcol]]=Train_df[divdict[divcol]].div(Train_df[divcol],axis=0).values

In [15]:
Train_df[loglist]=np.log(Train_df[loglist])

In [16]:
Train_df[minmaxnormlist]=(Train_df[minmaxnormlist]-Train_df[minmaxnormlist].min())/(Train_df[minmaxnormlist].max()-Train_df[minmaxnormlist].min())

In [17]:
# ORDTickDataname='GDF2022_4_12_19_51.csv'
ORDTickDataname='GDF_ORD2022_10_3_16_33.csv'
ORDTickGDF=pd.read_csv('/Data/LOBData/CollectedTickData/'+ORDTickDataname)

In [18]:
OTGDF=ORDTickGDF[(ORDTickGDF['TM_GP']>=0)&(ORDTickGDF['TM_GP']<39)]
OTGDF=OTGDF[OTGDF['MODCANCL_TP_CD']!=2]
OTGDF=OTGDF[OTGDF["ISU_CD"].isin(ISU_list)]

# OTGDF['NET_ORD_QTY']=OTGDF['ORD_QTY']*(2*OTGDF['ASKBID_TP_CD']-3)*(2-OTGDF['MODCANCL_TP_CD'])
OTGDF['NET_ORD_QTY']=OTGDF['ORD_QTY']*(2*OTGDF['ASKBID_TP_CD']-3)#'norm6feat
# OTGDF['NET_ORD_QTY']=OTGDF['ORD_QTY']*(2*OTGDF['ASKBID_TP_CD']-3)*OTGDF['ORD_PRC']#'norm6featNewPrc'
# OTGDF=OTGDF.set_index(['ORD_DD','ISU_CD','TM_GP'])


In [19]:
OTGDF

,Unnamed: 0,ORD_DD,ISU_CD,MBR_NO,BRN_NO,ASKBID_TP_CD,MODCANCL_TP_CD,ORD_ACPT_TM,ORD_QTY,ORD_PRC,TM_GP,ORD_VOL,NET_ORD_QTY
0,0,20150602,KR7018700005,2,29,1,1,NaN,605,0.000000,3.0,0.0,-605
9,9,20150602,KR7018700005,2,83,1,1,NaN,24467,757.857143,0.0,20055092.0,-24467
10,10,20150602,KR7018700005,2,83,1,3,NaN,3000,748.000000,0.0,2244000.0,-3000
11,11,20150602,KR7018700005,2,83,2,1,NaN,2952,741.600000,0.0,2196844.0,2952
12,12,20150602,KR7018700005,2,83,2,3,NaN,310,731.000000,0.0,227770.0,310
...,...,...,...,...,...,...,...,...,...,...,...,...,...
659377,3100,20151230,KR7058820002,56,17990,2,1,NaN,500,2095.000000,5.0,1047000.0,500
659379,3102,20151230,KR7058820002,56,17990,2,1,NaN,100,2090.000000,7.0,209000.0,100
659380,3103,20151230,KR7058820002,56,17990,2,1,NaN,200,2077.500000,9.0,415500.0,200
659383,3106,20151230,KR7058820002,56,17990,1,1,NaN,813,2115.000000,20.0,1719495.0,-813


In [20]:
groupcolumns=['ORD_DD', 'ISU_CD', 'MBR_NO', 'BRN_NO','TM_GP']
sumcolumns=['NET_ORD_QTY']
meancolumns,lastcolumns=[],[]

In [21]:
OTGDF=GetGroupDataFrame(OTGDF,groupcolumns,sumcolumns,meancolumns,lastcolumns)

feat_cols=['ASK_STEP1_BSTORD_RQTY', 'ASK_STEP2_BSTORD_RQTY', 
       'ASK_STEP3_BSTORD_RQTY',  'ASK_STEP4_BSTORD_RQTY', 
       'ASK_STEP5_BSTORD_RQTY', 'BID_STEP1_BSTORD_RQTY', 
       'BID_STEP2_BSTORD_RQTY', 'BID_STEP3_BSTORD_RQTY', 
       'BID_STEP4_BSTORD_RQTY','BID_STEP5_BSTORD_RQTY',
        '매도5단계호가합계잔량', '매수5단계호가합계잔량', '매도10단계호가합계잔량',
       '매수10단계호가합계잔량', '매도총호가잔량', '매수총호가잔량', '고가', '저가',
           '시가', '직전체결가격','NET_ORD_QTY']

feat_cols=['매도총호가잔량', '매수총호가잔량', '고가', '저가', 
           '직전체결가격','NET_ORD_QTY']

In [22]:
OTGDF

,ORD_DD,ISU_CD,MBR_NO,BRN_NO,TM_GP,NET_ORD_QTY
0,20150602,KR7018700005,2,29,3.0,-605
1,20150602,KR7018700005,2,83,0.0,-24205
2,20150602,KR7018700005,2,83,1.0,-1708
3,20150602,KR7018700005,2,83,2.0,-23805
4,20150602,KR7018700005,2,83,3.0,-365
...,...,...,...,...,...,...
254244,20151230,KR7058820002,56,17990,5.0,-300
254245,20151230,KR7058820002,56,17990,7.0,100
254246,20151230,KR7058820002,56,17990,9.0,200
254247,20151230,KR7058820002,56,17990,20.0,-813


In [23]:
feat_cols=['매도5단계호가합계잔량', '매수5단계호가합계잔량', '매도10단계호가합계잔량',
       '매수10단계호가합계잔량', '매도총호가잔량', '매수총호가잔량', '고가', '저가',
           '시가', '직전체결가격','NET_ORD_QTY']

In [24]:
mbrnlist = [(25,  1093),
            (12,   100),
            (17,    29),
            (44,     1),
            (42,     1),
            ( 2,   155),
            ( 2,    83),
            ( 4,  9997),
            (56, 17990),
            ( 5,   194)]

In [25]:
Train_df

,ORD_DD,ISU_CD,TM_GP,호가장처리가격,ASK_STEP1_BSTORD_PRC,ASK_STEP1_BSTORD_RQTY,ASK_STEP2_BSTORD_PRC,ASK_STEP2_BSTORD_RQTY,ASK_STEP3_BSTORD_PRC,ASK_STEP3_BSTORD_RQTY,...,매도총호가잔량,매수총호가잔량,ORD_PRC,시가,고가,저가,누적체결수량,누적거래대금,직전체결가격,10단계호가합계잔량
0,20150601,KR7018700005,0.0,764.558824,759.622549,3.217387e+03,761.392157,6307.411765,762.730392,6981.828431,...,0.032512,0.009845,723.901961,762.0,0.713965,0.715253,836.666667,6.356706e+05,0.709484,1.879948e+05
1,20150601,KR7018700005,1.0,744.210084,741.613445,8.523895e+04,743.336134,23937.075630,744.596639,6167.151261,...,0.033446,0.004851,695.336134,762.0,0.714083,0.712358,1648.319328,1.215550e+06,0.706583,2.280359e+05
2,20150601,KR7018700005,2.0,738.924370,729.462185,1.594209e+05,731.579832,5576.226891,732.621849,4160.218487,...,0.038091,0.001983,695.815126,762.0,0.714083,0.711302,1117.033613,8.143224e+05,0.704527,2.733246e+05
3,20150601,KR7018700005,3.0,731.469484,727.126761,1.422302e+06,728.248826,11210.802817,729.248826,2467.173709,...,0.107669,0.000036,718.826291,762.0,0.714083,0.711302,266.732394,1.939907e+05,0.704119,1.472573e+06
4,20150601,KR7018700005,4.0,727.269231,727.000000,1.657631e+06,728.000000,11533.591512,729.000000,204.082228,...,0.123010,0.000000,726.305040,762.0,0.714083,0.711302,10.094164,7.338458e+03,0.704097,1.705071e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57715,20151230,KR7071930002,34.0,2106.666667,2118.333333,7.366667e+01,2123.333333,43.333333,2128.333333,10.000000,...,0.000330,0.000274,2106.666667,0.0,0.838928,0.848730,66.666667,1.406667e+05,0.841047,2.049000e+03
57716,20151230,KR7071930002,35.0,2106.666667,2118.333333,7.366667e+01,2123.333333,43.333333,2128.333333,10.000000,...,0.000330,0.000274,2106.666667,0.0,0.838928,0.848730,66.666667,1.406667e+05,0.841047,2.049000e+03
57717,20151230,KR7071930002,36.0,2106.666667,2118.333333,7.366667e+01,2123.333333,43.333333,2128.333333,10.000000,...,0.000330,0.000274,2106.666667,0.0,0.838928,0.848730,66.666667,1.406667e+05,0.841047,2.049000e+03
57718,20151230,KR7071930002,37.0,2106.666667,2118.333333,7.366667e+01,2123.333333,43.333333,2128.333333,10.000000,...,0.000330,0.000274,2106.666667,0.0,0.838928,0.848730,66.666667,1.406667e+05,0.841047,2.049000e+03


In [26]:
if Train_df['직전체결가격'].isnull().any():
    raise ValueError

In [27]:
A=Train_df[Train_df['직전체결가격'].isnull()]

In [28]:
set(A['ISU_CD'])

set()

In [29]:
aaaa

NameError: name 'aaaa' is not defined

In [30]:
for mbr,brn in mbrnlist:
    print(mbr,brn)
    OTGDF_temp=OTGDF[(OTGDF['MBR_NO']==mbr) &(OTGDF['BRN_NO']==brn)]
    OTGDF_temp=OTGDF_temp.set_index(['ORD_DD','ISU_CD','TM_GP'])
    OTGDF_temp=OTGDF_temp.reindex(new_index)
    OTGDF_temp['NET_ORD_QTY']=OTGDF_temp['NET_ORD_QTY'].fillna(0)
    Train_df['NET_ORD_QTY']=OTGDF_temp['NET_ORD_QTY'].values
    Train_df['ylabel']=Train_df['NET_ORD_QTY']
    Train_df['ylabel'][:-1]=(2*(Train_df['NET_ORD_QTY']>0)+1*(Train_df['NET_ORD_QTY']==0))[1:]
    Train_df['ylabel'][38::39]=1
    print(Train_df['NET_ORD_QTY'].sum())    
    SaveDirPath='/Data/LOBData/TrainData/'
    now=datetime.now()
    DataSubfix=str(mbr)+'_'+str(brn)+Dataname
    TrainDataname='Train_ORD'+'_'+DataSubfix
    TrainLabelDataname='Train_ORD_Label_'+'_'+DataSubfix

    np.save(SaveDirPath+TrainDataname,Train_df[feat_cols].values)
    np.save(SaveDirPath+TrainLabelDataname,Train_df['ylabel'])
    print("path:",SaveDirPath+TrainDataname)
    print("path:",SaveDirPath+TrainLabelDataname)
    

25 1093
-114500.0
path: /Data/LOBData/TrainData/Train_ORD_25_10931516Train
path: /Data/LOBData/TrainData/Train_ORD_Label__25_10931516Train
12 100
39474994.0
path: /Data/LOBData/TrainData/Train_ORD_12_1001516Train
path: /Data/LOBData/TrainData/Train_ORD_Label__12_1001516Train
17 29


/root/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/root/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


-5568193.0
path: /Data/LOBData/TrainData/Train_ORD_17_291516Train
path: /Data/LOBData/TrainData/Train_ORD_Label__17_291516Train
44 1
-11881781.0
path: /Data/LOBData/TrainData/Train_ORD_44_11516Train
path: /Data/LOBData/TrainData/Train_ORD_Label__44_11516Train
42 1
-5753619.0
path: /Data/LOBData/TrainData/Train_ORD_42_11516Train
path: /Data/LOBData/TrainData/Train_ORD_Label__42_11516Train
2 155
21723773.0
path: /Data/LOBData/TrainData/Train_ORD_2_1551516Train
path: /Data/LOBData/TrainData/Train_ORD_Label__2_1551516Train
2 83
4090950.0
path: /Data/LOBData/TrainData/Train_ORD_2_831516Train
path: /Data/LOBData/TrainData/Train_ORD_Label__2_831516Train
4 9997
8999208.0
path: /Data/LOBData/TrainData/Train_ORD_4_99971516Train
path: /Data/LOBData/TrainData/Train_ORD_Label__4_99971516Train
56 17990
32329000.0
path: /Data/LOBData/TrainData/Train_ORD_56_179901516Train
path: /Data/LOBData/TrainData/Train_ORD_Label__56_179901516Train
5 194
33402976.0
path: /Data/LOBData/TrainData/Train_ORD_5_1941516

In [33]:
(Train_df['ylabel']-1).sum()

3084.0

In [34]:
with open(SaveDirPath+'DataInformationORD.txt', "a+") as f:
    f.write("TrainDataname:"+TrainDataname+'\n')
    f.write("TrainLabelDataname:"+TrainLabelDataname+'\n')
    f.write("TickDataname:"+TickDataname+'\n')
    f.write("ORDTickDataname:"+ORDTickDataname+'\n')

    f.write("divdict:"+'\n')
    for col,llist in divdict.items():
        f.write(str(col)+'\t'+str(llist))
    f.write('\n')
    f.write("loglist:"+'\n')
    for col in loglist:
        f.write(str(col)+'\t')
    f.write('\n')

    f.write("feat_cols:"+'\n')
    for col in feat_cols:
        f.write(str(col)+'\t')
    f.write('\n')